In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
limp = lazy_imports()

import pandas as pd
import numpy as np
from dask import dataframe as dd
from pyspark import SparkContext
from openpyxl import load_workbook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import dash
import bokeh
import altair as alt
import pydot
import statistics
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
import keras
import nltk
import gensim
import spacy
import re
import sys
import os
import glob
from pathlib import Path
import pickle
import datetime as 

In [3]:
# Read in training data
train_data = Path("data/train.csv")
train_df = pd.read_csv(train_data, delimiter=",", index_col='Id')
train_df = train_df.fillna('0')
train_df = pd.get_dummies(train_df)
train_df.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,8450,7,5,2003,2003,706,0,150,856,...,0,0,0,1,0,0,0,0,1,0
2,20,9600,6,8,1976,1976,978,0,284,1262,...,0,0,0,1,0,0,0,0,1,0
3,60,11250,7,5,2001,2002,486,0,434,920,...,0,0,0,1,0,0,0,0,1,0
4,70,9550,7,5,1915,1970,216,0,540,756,...,0,0,0,1,1,0,0,0,0,0
5,60,14260,8,5,2000,2000,655,0,490,1145,...,0,0,0,1,0,0,0,0,1,0


In [4]:
# Create the features (X) and target (y) sets
X = train_df.drop(['SalePrice'], axis=1)
y = train_df["SalePrice"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
# Creating StandardScaler instance
scaler = sklearn.preprocessing.StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
max_feat = round(len(train_df.columns)*.75)

In [8]:
max_feat

629

In [9]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    regressor = GradientBoostingRegressor(n_estimators=200,
                                            learning_rate=learning_rate,
                                            max_features=max_feat,
                                            max_depth=5,
                                            random_state=0)

    # Fit the model
    regressor.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.5f}".format(
        regressor.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (test): {0:.5f}".format(
        regressor.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.99271
Accuracy score (test): 0.90802

Learning rate:  0.1
Accuracy score (training): 0.99589
Accuracy score (test): 0.90214

Learning rate:  0.25
Accuracy score (training): 0.99906
Accuracy score (test): 0.88688

Learning rate:  0.5
Accuracy score (training): 0.99993
Accuracy score (test): 0.83900

Learning rate:  0.75
Accuracy score (training): 0.99999
Accuracy score (test): 0.77818

Learning rate:  1
Accuracy score (training): 1.00000
Accuracy score (test): 0.71397



In [15]:
# Choose a learning rate and create classifier
regressor = GradientBoostingRegressor(n_estimators=200,
                                        learning_rate=0.25,
                                        max_features=5,
                                        max_depth=5,
                                        random_state=0)

# Fit the model
regressor.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = regressor.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).tail(20)

,Prediction,Actual
345,245553.414684,260400
346,179119.165692,175000
347,119563.755715,141500
348,149156.228397,134900
349,184153.972691,219500
350,294449.258651,266000
351,141054.170421,143000
352,136139.458357,132000
353,165708.616655,180500
354,106967.036121,129900


In [56]:
test_data = Path("data/test.csv")
test_df = pd.read_csv(test_data, delimiter=",", index_col='Id')
test_df = train_df.fillna('0')
test_df = pd.get_dummies(train_df)
test_df.head()
test_df.drop(['SalePrice'], axis=1, inplace=True)

In [55]:
test_df

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,8450,7,5,2003,2003,706,0,150,856,...,0,0,0,1,0,0,0,0,1,0
2,20,9600,6,8,1976,1976,978,0,284,1262,...,0,0,0,1,0,0,0,0,1,0
3,60,11250,7,5,2001,2002,486,0,434,920,...,0,0,0,1,0,0,0,0,1,0
4,70,9550,7,5,1915,1970,216,0,540,756,...,0,0,0,1,1,0,0,0,0,0
5,60,14260,8,5,2000,2000,655,0,490,1145,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,7917,6,5,1999,2000,0,0,953,953,...,0,0,0,1,0,0,0,0,1,0
1457,20,13175,6,6,1978,1988,790,163,589,1542,...,0,0,0,1,0,0,0,0,1,0
1458,70,9042,7,9,1941,2006,275,0,877,1152,...,0,0,0,1,0,0,0,0,1,0


In [35]:
test_scaled = X_scaler.transform(test_df)

In [60]:
# Make Prediction
final_predictions = regressor.predict(test_scaled)
final = pd.DataFrame({"id":test_df.index,"SalePrice": final_predictions})

In [49]:
final

,id,SalePrice
0,1,201864.783706
1,2,170992.542169
2,3,225698.991080
3,4,151524.271580
4,5,262537.218572
...,...,...
1455,1456,179119.165692
1456,1457,200368.578410
1457,1458,273094.648453
1458,1459,142004.398858


In [53]:
final.to_csv('submission.csv', index= False)